In [ ]:
# Most files do not seem to have recordtype 30 in them.  It would be useful to find an example of a file that did 
from bs4 import BeautifulSoup

with open("download_html/os_download_links_page.htm") as f:
    soup = BeautifulSoup(f.read())

soup.find("a")

el = soup.find(text = "Number of Files:").parent.parent
num = el.text.replace("Number of Files:", "").strip()
numfiles = int(num)

my_links = set()
for a in soup.findAll("a"):
    if "href" in a.attrs:
        if "AB76DL" in a["href"]:  # This is a bit of trial and error, but it turned out that the download links all contain "AB76DL" as part of the URL
            my_links.add(a["href"])
my_links = list(my_links)

# Check we've found the right number of links
if (len(my_links) != numfiles):
    raise Exception

In [ ]:
# For py 2 and 3 compataibility
try:
    import urllib.request as urlrequest
except ImportError:
    import urllib as urlrequest
    
import re

import requests, zipfile, io


In [ ]:
counter = -1
for my_link in my_links:
    counter +=1
    if counter % 50 == 0:
        print(f"counter {counter} got to file {my_link}")
    r = requests.get(my_link)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()

    contents = z.filelist[0]  #assume there is just one file.  Check it's a csv else error
    file_bytes = z.open(contents).read()


    lines = file_bytes.decode("utf-8").split("\r\n")

    lines_30 = [l for l in lines if l[0:2] == '30']
    
    if len(lines_30) > 0:
        print(f"found link {my_link}")
        break